In [1]:
import pandas

In [2]:
train = pandas.read_csv("A:/Programming/Python/Data/AI_DOCTOR/Any_NLP_Problem/spooky/train.csv")
test  = pandas.read_csv("A:/Programming/Python/Data/AI_DOCTOR/Any_NLP_Problem/spooky/test.csv")
sample= pandas.read_csv("A:/Programming/Python/Data/AI_DOCTOR/Any_NLP_Problem/spooky/sample_submission.csv")

In [3]:
train.head()

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [4]:
test.head()

,id,text
0,id02310,"Still, as I urged our leaving Ireland with suc..."
1,id24541,"If a fire wanted fanning, it could readily be ..."
2,id00134,And when they had broken down the frail door t...
3,id27757,While I was thinking how I should possibly man...
4,id04081,I am not sure to what limit his knowledge may ...


In [5]:
sample.head()

,id,EAP,HPL,MWS
0,id02310,0.403494,0.287808,0.308698
1,id24541,0.403494,0.287808,0.308698
2,id00134,0.403494,0.287808,0.308698
3,id27757,0.403494,0.287808,0.308698
4,id04081,0.403494,0.287808,0.308698


In [6]:
def multiclass_logloss(actual, predicted, eps=1e-15):
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2
    clip = np.clip(predicted, eps, 1-eps)
    rows = actual.shape[0]
    vsota = np.sum(actual*np.log(clip))
    return -1.0/rows*vsota

In [7]:
from sklearn import preprocessing

In [8]:
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(train.author.values)

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
x_train, x_valid, y_train, y_valid = train_test_split(train.text.values, y,
                                                     stratify=y,
                                                     test_size=0.1,
                                                     random_state=42,
                                                     shuffle=True)

In [11]:
print("x_train.shape", x_train.shape)
print("x_valid.shape", x_valid.shape)

x_train.shape (17621,)
x_valid.shape (1958,)


In [12]:
17621 + 1958

19579

In [13]:
y.shape

(19579,)

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [15]:
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

tfv.fit(list(x_train) + list(x_valid))

x_train_tfv = tfv.transform(x_train)
x_valid_tfv = tfv.transform(x_valid)

In [16]:
from sklearn.linear_model import LogisticRegression

In [17]:
import numpy as np

In [18]:
#TF-IDF with Logistic Regression

In [19]:
clf = LogisticRegression(C=0.1)
clf.fit(x_train_tfv, y_train)
predictions_tfidf_logisticregression = clf.predict_proba(x_valid_tfv)

In [20]:
print("Multiclass log loss for \n TF-IDF with logistic regression: %0.3f" % multiclass_logloss(y_valid, predictions_tfidf_logisticregression))

Multiclass log loss for 
 TF-IDF with logistic regression: 0.854


In [21]:
#Count Vectorizer with logistic regression

In [22]:
ctv = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), stop_words = 'english')
ctv.fit(list(x_train) + list(x_valid))

x_train_ctv = ctv.transform(x_train)
x_valid_ctv = ctv.transform(x_valid)

In [23]:
clf = LogisticRegression(C=0.1)
clf.fit(x_train_ctv, y_train)
predictions = clf.predict_proba(x_valid_ctv)

In [24]:
print("Multiclass log loss for \n CountVectorizer with logistic regression %0.3f"% multiclass_logloss(y_valid, predictions))

Multiclass log loss for 
 CountVectorizer with logistic regression 0.608


In [25]:
#Fitting a naive bayes classifier on TF-IDF

In [26]:
from sklearn.naive_bayes import MultinomialNB

In [27]:
clf = MultinomialNB()
clf.fit(x_train_tfv, y_train)
predictions = clf.predict_proba(x_valid_tfv)

print ("logloss: %0.3f " % multiclass_logloss(y_valid, predictions))

logloss: 0.578 


In [28]:
#Fitting a naive bayes classifier on Count Vectorizer

In [29]:
clf = MultinomialNB()
clf.fit(x_train_ctv, y_train)
predictions_multinomial_ct = clf.predict_proba(x_valid_ctv)

print ("logloss: %0.3f " % multiclass_logloss(y_valid, predictions_multinomial_ct))

logloss: 0.485 


In [31]:
from sklearn import decomposition

In [32]:
svd = decomposition.TruncatedSVD(n_components=120)
svd.fit(x_train_tfv)
x_train_svd = svd.transform(x_train_tfv)
x_valid_svd = svd.transform(x_valid_tfv)

# Scale the data obtained from SVD. Renaming variable to reuse without scaling.
scl = preprocessing.StandardScaler()
scl.fit(x_train_svd)
x_train_svd_scl = scl.transform(x_train_svd)
x_valid_svd_scl = scl.transform(x_valid_svd)

In [34]:
from sklearn.svm import SVC

In [35]:
clf = SVC(C=1.0, probability=True) # since we need probabilities
clf.fit(x_train_svd_scl, y_train)
predictions = clf.predict_proba(x_valid_svd_scl)

print ("logloss: %0.3f " % multiclass_logloss(y_valid, predictions))

logloss: 0.739 


In [36]:
# Fitting xgboost on tfidf

In [37]:
import xgboost as xgb

In [38]:
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(x_train_tfv.tocsc(), y_train)
predictions = clf.predict_proba(x_valid_tfv.tocsc())

print ("logloss: %0.3f " % multiclass_logloss(y_valid, predictions))

logloss: 0.782 


In [39]:
# Fitting xgboost on count vectorizer

In [41]:
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(x_train_ctv.tocsc(), y_train)
predictions = clf.predict_proba(x_valid_ctv.tocsc())

print ("logloss: %0.3f " % multiclass_logloss(y_valid, predictions))

logloss: 0.772 


In [ ]:
# Fitting xgboost on tfidf svd features

In [43]:
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(x_train_svd, y_train)
predictions = clf.predict_proba(x_valid_svd)

print ("logloss: %0.3f " % multiclass_logloss(y_valid, predictions))

logloss: 0.765 


In [45]:
clf = xgb.XGBClassifier(nthread=10)
clf.fit(x_train_svd, y_train)
predictions = clf.predict_proba(x_valid_svd)

print ("logloss: %0.3f " % multiclass_logloss(y_valid, predictions))

logloss: 0.792 
